Ref: 
1. https://github.com/langchain-ai/langchain/blob/master/cookbook/rag_fusion.ipynb?ref=blog.langchain.dev
2. https://github.com/Raudaschl/rag-fusion

RAG-Fusion, a search methodology that aims to bridge the gap between traditional search paradigms and the multifaceted dimensions of human queries.  

Inspired by the capabilities of Retrieval Augmented Generation (RAG), this project goes a step further by employing `multiple query generation` and `Reciprocal Rank Fusion` to re-rank search results.

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

import rich

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

local_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
all_documents = {
    "doc1": "Climate change and economic impact.",
    "doc2": "Public health concerns due to climate change.",
    "doc3": "Climate change: A social perspective.",
    "doc4": "Technological solutions to climate change.",
    "doc5": "Policy changes needed to combat climate change.",
    "doc6": "Climate change and its impact on biodiversity.",
    "doc7": "Climate change: The science and models.",
    "doc8": "Global warming: A subset of climate change.",
    "doc9": "How climate change affects daily weather.",
    "doc10": "The history of climate change activism.",
}

#### Using Chroma

In [ ]:
from langchain_community.vectorstores import Chroma
# vectorstore = Chroma.from_texts(all_documents.values(), embedding=OpenAIEmbeddings(), collection_name="rag_fusion", persist_directory="./rag_fusion_db")
vectorstore = Chroma.from_texts(all_documents.values(), embedding=local_embeddings, collection_name="rag_fusion", persist_directory="./rag_fusion_db")

#### Using FAISS

In [ ]:
# from langchain.vectorstores.faiss import FAISS
# vectorstore = FAISS.from_texts(all_documents.values(), embedding=OpenAIEmbeddings())

## Define the Query Generator

In [ ]:
from langchain import hub

prompt = hub.pull("langchain-ai/rag-fusion-query-generation")
rich.print(prompt)

In [ ]:
# def split_lines(x):
#     return x.split("\n")

# generate_queries = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.5) | StrOutputParser() | split_lines

generate_queries = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.5) | StrOutputParser() | (lambda x: x.split("\n"))

## Define the full chain
We can now put it all together and define the full chain.
1. Generate a bunch queries
2. Retrieval by each query
3. Joins all the results and ordered by Reciprocal Rank Fusion

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.load import dumps, loads

def rrf(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # assumes the docs are returned in the order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1/(rank+k)

    reranked_results = [
        (loads(doc_str), score) for doc_str, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    
    return reranked_results

In [ ]:
chain = generate_queries | retriever.map() | rrf

In [ ]:
original_queries = "Impact of climat change"
final_results = chain.invoke({"original_query": original_queries})

In [ ]:
rich.print(final_results)